In [ ]:
import face_recognition
import cv2
import numpy as np
from math import sin, cos, radians
import pickle
import os
import math

In [ ]:
video_capture = cv2.VideoCapture(0)
# Test if there are already files to load
# In case there is, it will just load
# Otherwise, a base will be created, converted to bits and loaded into the script
ArquivoEncoding = "Codings.p"
ArquivoNome = "Names.p"
known_face_encodings = []
known_face_names = []

if not(os.path.isfile("Codings.p")): #if the file does not exist
    with open(ArquivoEncoding, 'wb') as fp:
        pickle.dump(known_face_encodings, fp)
else:
    with open (ArquivoEncoding, 'rb') as fp:
        print("data loaded")
        known_face_encodings = pickle.load(fp)

if not(os.path.isfile("Names.p")):
    with open(ArquivoNome, 'wb') as fp:
        pickle.dump(known_face_names, fp)
else:
    with open (ArquivoNome, 'rb') as fp:
        known_face_names = pickle.load(fp)


In [ ]:
def face_distance_to_conf(face_distance, face_match_threshold=0.6):
    if face_distance > face_match_threshold:
        range = (1.0 - face_match_threshold)
        linear_val = (1.0 - face_distance) / (range * 2.0)
        return linear_val
    else:
        range = face_match_threshold
        linear_val = 1.0 - (face_distance / (range * 2.0))
        return linear_val + ((1.0 - linear_val) * math.pow((linear_val - 0.5) * 2, 0.2))

In [ ]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
confidence = " xx.x"
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]

            '''
            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
#             print(best_match_index)
#             print(len(matches))
            if matches[best_match_index%len(matches)]:
                name = known_face_names[best_match_index]
#                 confidence = face_distance_to_conf(face_distances)
            '''
            face_names.append(name)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name+confidence, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
#         cv2.putText(frame, confidence, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Press 'c' to capture face and assign a name to it
    if cv2.waitKey(33) & 0xFF == ord('c'):
        # Capture the picture
        ret, frame = video_capture.read()
        frameRGB = frame[:,:,::-1]
        # Recognize and store face's measures
        known_face_encodings.append(face_recognition.face_encodings(frameRGB)[0])
        # Assign a name to the captured face
        nome = input("Type captured face's name: ")
        known_face_names.append(nome)
        # Save the image for future use
#         cv2.imwrite('opencv '+nome+'.png', frame)
        
     # Press 's' to save everything into bits
    if cv2.waitKey(33) & 0xFF == ord('s'):
        if bool(input("Do you wish to save all the names and encodings? (1|0):\n")):
            with open(ArquivoEncoding, 'wb') as fp:
                pickle.dump(known_face_encodings, fp)
            with open (ArquivoEncoding, 'rb') as fp:
                known_face_encodings = pickle.load(fp)
            with open(ArquivoNome, 'wb') as fp:
                pickle.dump(known_face_names, fp)
            with open (ArquivoNome, 'rb') as fp:
                known_face_names = pickle.load(fp)
            print("Names' and encodings' lists converted in binary and saved!")

        
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()
